# News headlines in alternative universes
Project for the  Data Jam for Newspaper Navigator library of congress.

## UPDATE:
I recommend to open directly the colab version instead of the jupyter notebook version: https://colab.research.google.com/drive/1F0FZfoS_cbi1C-vaaln-DtHW5m554z2a?usp=sharing                                                             

Proposal: We choose a year in which a world tragedy happened. We select headlines related to that tragey and train an algorithm to learn to create fake headlines using the style and prhases of that year. The algorithm takes a user generated phrase negating the tragedy and completes it as if those tragedies didn't occur.

We test with 1910, that year the mexican revolution started because Porfirio Diaz was in power for more than 31 years.
 
    Input phrase:  'Peace in Mexico after Porfirio Diaz is removed from office.'

    Fake headline from 1910: 'Peace in Mexico after Porfirio Diaz is removed from office.\nJAPAN WILL SEND FLEET Mexico is Elated Over Pre parations Made to Cele brate Independence.'

    Fake headline from 1910: 'Peace in Mexico after Porfirio Diaz is removed from office. By - the Mexican News Agency \' River Review.\nSTATUS OF MEXICO IMPROV ENOUGH FOR NOW El Paso Residents Opt- In to Stay put and Watch the Movements in Mexico.'


    Input phrase:'Porfirio Diaz stepped out of office'
    
    Fake headline from 1910: 'Porfirio Diaz stepped out of office today, becoming President of Mexico for the fifth time'    

    Input phrase:'Porfirio Diaz stepped out of office to avoid a war'
    
    Fake headline from 1910: 'Porfirio Diaz stepped out of office to avoid a war, but American officials were fearful that an armed conflict was imminent'


Data used:
    
    1910_headlines.json, 1910 is the year of the Mexican Revolution
      
Tools used:
    
    python3 
    Google colab notegook on GPT-2 (while it uses tensorflow, no tensorflow knowledge is required to reproduce this experiment).
    

Data collection.
To test you can download a sample data from 'https://news-navigator.labs.loc.gov/prepackaged/1910_headlines_sample.json'
Instructions to get the whole dataset from the same of different years can be found at https://news-navigator.labs.loc.gov


In [ ]:
# Eric Dolores, library of congress open data

year = '1910' #include the year you want to obtaine the headlines from
assert year 

os.system(f'wget https://news-navigator.labs.loc.gov/prepackaged/{year}_headlines.json')


Once the json file is downloaded we need:
    
    the path to that json file
    keys: a list of topics we are interested in. 
    badWord/badPairOfWords: a word or a pair of words that may be included due to the similarity to the topics but is/are unrelated, we will use it to remove non related headlines
    

In [ ]:

# include the place where the json is saved in your local computer
pathToJsonHeadlines = f'{year}_headlines.json' #change this by the name/location of the file you downloaded
keys = ['Mexico','Mexican'] # modify this to include the list of topics, if not topics are given you will work with the whole year of data
# you can include at most one of the next two variables or write your custome removal code for several unrelated words
pairOfBadWords ='New Mexico'
badWord = ''


The following code:

- reads the headlines
- filters by the keys
- removes unrelated headlines
- saves into a file

In [ ]:
 Eric Dolores, library of congress open data

import json

assert pathToJsonHeadlines #include the place where the json is saved in your local computer

with open(pathToJsonHeadlines,'r') as f:
    jsonData = json.load(f)
    
print(f'sample dictionary:\n {jsonData[0]}\n') 

def pairRemoval(allHeadlines, badPairOfWords):
    filtered = []
    begining = badPairOfWords.split(' ')[0]
    end = badPairOfWords.split(' ')[1]
    for stuff in allHeadlines:
        for i, item in enumerate(stuff['ocr']):
            if item == end and i>0 and stuff['ocr'][i-1] ==begining:
                break
        filtered.append(stuff)
    print(f'we removed those headlines containing {badPairOfWords} and now we have {len(filtered)} headlines \n')
    return filtered

def wordRemoval(allHeadlines, badWord):
    filtered = []
    for stuff in allHeadlines:
        if item in stuff['ocr']:
                continue
        filtered.append(stuff)    
    print(f'we removed those headlines containing {badWord} and now we have {len(filtered)} headlines \n')
    return filtered


print('We the data and keep  those that use the key words, if any.')
if keys:

  # filter by the keywords
  relatedHeadlines = []
  for element in jsonData:
      for key in keys:
          if key in element['ocr']:
              relatedHeadlines.append(element)
              break
            
  print(f'we found {len(relatedHeadlines)} headlines having the key words {keys} \n')

  # We filter for wrong headlines, perhaps you want Mexico but not New Mexico
else:
  relatedHeadlines = jsonData

print('Now we remove words related to topics we want to avoid')
if pairOfBadWords:
    filtered = pairRemoval(relatedHeadlines, pairOfBadWords) # I needed it to remove 'New Mexico' while looking for the country 'Mexico'
elif badWord:
    filtered = wordRemoval(relatedHeadlines, badWord)
else:
    filtered = relatedHeadlines


print(f'sample filtered input:\n {filtered[0]}') 

#We purpusedly delayed the construction of strings in case the metadata is useful
strings = ''
for item in filtered:
    prhase = ' '.join(item['ocr'])
    strings = strings + prhase+' \n '

# We collect those strings
filename=pathToJsonHeadlines.replace('json','txt')    

with open(filename,'w') as f:
    f.write(strings)     
print(f"We saved the input into the file {filename}")


### Conclusions and future ways to continue this project:
This project was born on the Data Jam for Newspaper Navigator library of congress. I realized early that training was going to consume most of the time.

This is very biased data,  a revolution started in 1910 so random strings are related to war in Mexico:

    NO PEACE FOR MEXICO CITY Government to Be Quieter in the Postponed Marshal Correa. President Too Escapes to Attend United Nations Conference. 
 

Even if we give a guideline, the algorithm may not return a headline with the output we aim:


    Input phrase:'Porfirio Diaz stepped out of office'

    Fake headline from 1910: 'Porfirio Diaz stepped out of office today, becoming President of Mexico for the fifth time' 


In fact the algorithm returns a headline given an arbitrary input.

    Input phrase: 'Burritos are declared a danger to society '

    Fake headline from 1910: 'Burritos are declared a danger to society and the state militia is sent out to quell the uprising.' 

 
 
Since we have a dataset of images of 1910, one possible continuation of this project is to assign an image to the fake headline and return a fake news paper first plane. I haven't found a natural way to associate a picture file to an arbitrary string but as soon as a good enough solution occurs I'll write that part.
 
 
 
 


We rely on the work of Max Woolf http://minimaxir.com/ and the data from the Library of Congress, Newspaper Navigator dataset: Extracted Visual Content from Chronicling America. https://news-navigator.labs.loc.gov